In [1]:
import cv2
import numpy as np
from keras.models import load_model

# Load the trained model
model = load_model('fer2013plus.h5')

# Define emotion labels
emotion_labels = ["Anger", "Contempt", "Disgust", "Fear", "Happiness", "Neutral", "Sadness", "Surprise"]

# Load the Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start the video capture
cap = cv2.VideoCapture(0)  # Use 0 for the default camera, you may need to change it based on your setup

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # If faces are detected, process each face
    for (x, y, w, h) in faces:
        # Crop the face region
        face_roi = gray_frame[y:y + h, x:x + w]

        # Resize the face region to 48x48 pixels
        resized_face = cv2.resize(face_roi, (48, 48))
        
        # Normalize pixel values to be in the range [0, 1]
        normalized_face = resized_face / 255.0

        # Add a batch dimension and channel dimension to the face
        processed_face = np.expand_dims(np.expand_dims(normalized_face, axis=-1), axis=0)

        # Make predictions using the model
        predictions = model.predict(processed_face)
        
        # Draw a rectangle around the face and display the predicted emotion
        predicted_emotion_index = np.argmax(predictions)
        predicted_emotion = emotion_labels[predicted_emotion_index]

        # Draw a rectangle around the face and display the predicted emotion
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Emotion Detection', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()





1/1 [==============================] - 0s 29ms/step
